# HW 3: Neural Machine Translation

In this homework you will build a full neural machine translation system using an attention-based encoder-decoder network to translate from German to English. The encoder-decoder network with attention forms the backbone of many current text generation systems. See [Neural Machine Translation and Sequence-to-sequence Models: A Tutorial](https://arxiv.org/pdf/1703.01619.pdf) for an excellent tutorial that also contains many modern advances.

## Goals


1. Build a non-attentional baseline model (pure seq2seq as in [ref](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)). 
2. Incorporate attention into the baseline model ([ref](https://arxiv.org/abs/1409.0473) but with dot-product attention as in class notes).
3. Implement beam search: review/tutorial [here](http://www.phontron.com/slides/nlp-programming-en-13-search.pdf)
4. Visualize the attention distribution for a few examples. 

Consult the papers provided for hyperparameters, and the course notes for formal definitions.

This will be the most time-consuming assignment in terms of difficulty/training time, so we recommend that you get started early!

## Setup

This notebook provides a working definition of the setup of the problem itself. Feel free to construct your models inline, or use an external setup (preferred) to build your system.

In [0]:
!pip install -q torch torchtext spacy opt_einsum
!pip install -qU git+https://github.com/harvardnlp/namedtensor
!python -m spacy download en
!python -m spacy download de


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/de

    You can now load the model via spacy.load('de')



In [0]:
# Torch
import torch
# Text text processing library and methods for pretrained word embeddings
from torchtext import data, datasets
# Named Tensor wrappers
from namedtensor import ntorch, NamedTensor
from namedtensor.text import NamedField

We first need to process the raw data using a tokenizer. We are going to be using spacy, but you can use your own tokenization rules if you prefer (e.g. a simple `split()` in addition to some rules to acccount for punctuation), but we recommend sticking to the above.

In [0]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


Note that we need to add the beginning-of-sentence token `<s>` and the end-of-sentence token `</s>` to the 
target so we know when to begin/end translating. We do not need to do this on the source side.

In [0]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = NamedField(names=('srcSeqlen',), tokenize=tokenize_de)
EN = NamedField(names=('trgSeqlen',), tokenize=tokenize_en,
                init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

Let's download the data. This may take a few minutes.

While this dataset of 200K sentence pairs is relatively small compared to others, it will still take some time to train. We only expect you to work with sentences of length at most 20 for this homework. You are expected to train on at least this reduced dataset for this homework, but are free to experiment with the rest of the training set as well.

**We encourage you to start with `MAX_LEN=20` but encourage experimentation after getting reasonable results with the filtered data. The baseline scores are based on models train on the filtered data.**

In [0]:
MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

{'src': <namedtensor.text.torch_text.NamedField object at 0x7f5748f8a978>, 'trg': <namedtensor.text.torch_text.NamedField object at 0x7f5748f8a908>}
119076
{'src': ['David', 'Gallo', ':', 'Das', 'ist', 'Bill', 'Lange', '.', 'Ich', 'bin', 'Dave', 'Gallo', '.'], 'trg': ['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.']}


In [0]:
!head -n 1 .data/iwslt/de-en/train.de-en.de
!head -n 1 .data/iwslt/de-en/train.de-en.en

David Gallo: Das ist Bill Lange. Ich bin Dave Gallo.
David Gallo: This is Bill Lange. I'm Dave Gallo.


In [0]:
src = open("valid.src", "w")
trg = open("valid.trg", "w")
for example in val:
    print(" ".join(example.src), file=src)
    print(" ".join(example.trg), file=trg)
src.close()
trg.close()

Now we build the vocabulary and convert the text corpus into indices. We are going to be replacing tokens that occurred less than 5 times with `<unk>` tokens, and take the rest as our vocab.

In [0]:
MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

[('.', 113253), (',', 67237), ('ist', 24189), ('die', 23778), ('das', 17102), ('der', 15727), ('und', 15622), ('Sie', 15085), ('es', 13197), ('ich', 12946)]
Size of German vocab 13353
[('.', 113433), (',', 59512), ('the', 46029), ('to', 29177), ('a', 27548), ('of', 26794), ('I', 24887), ('is', 21775), ("'s", 20630), ('that', 19814)]
Size of English vocab 11560
2 3


Now we split our data into batches as usual. Batching for MT is slightly tricky because source/target will be of different lengths. Fortunately, `torchtext` lets you do this by allowing you to pass in a `sort_key` function. This will minimizing the amount of padding on the source side, but since there is still some padding you will inadvertendly "attend" to these padding tokens. 

One way to get rid of padding is to pass a binary `mask` vector to your attention module so its attention score (before the softmax) is minus infinity for the padding token. Another way (which is how we do it for some of our projects) is to manually sort data into batches so that each batch has exactly the same source length (this means that some batches will be less than the desired batch size, though).

However, for this homework padding won't matter too much, so it's fine to ignore it.

In [0]:
BATCH_SIZE = 32
device = torch.device('cuda:0')
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=device,
                                                  repeat=False, sort_key=lambda x: len(x.src))

Let's check to see that the BOS/EOS token is indeed appended to the target (English) sentence.

In [0]:
batch = next(iter(train_iter))
print("Source")
print(batch.src)
print("Target")
print(batch.trg)


Source
NamedTensor(
	tensor([[   77,    12,    28,    28,   820,    23,  7273,    20,   322,   221,
             0,   104,    77,    12,    17,   148,    77,    20,    97,   868,
            28,    99,    12,    23,    28,    12,   150,   112,    23,   804,
           612,   212],
        [    4,   236,    33,     2,    50,    35,   103,   679,  3154,    21,
          1008,     3,     4,    11,   473,    70,    47,    67,     9,    24,
           119,   528,   167,     4,    33,    44,   172,    30,     4,  2056,
             3,    33],
        [   29,  1294,   403,     1,    10,   409,    32,    51,     0,   624,
            10,   898,     6,   203,   188,   678,    13,   647,    32,  5581,
            53,  1191,    13,  1867,   114,  1337,    24,  4387,  4318,  1182,
            13,    13],
        [    7,   109,  6226,     1,  1040,   248,     5,  4650,     3,     3,
            32,    11,  7606,     3, 11713,    42,    94,    19,    18,   618,
          5737,     4,   292,     2,  

Success! Now that we've processed the data, we are ready to begin modeling.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/harvard-cs287-s19-hw3/

For the final Kaggle test, we will provide the source sentence, and you are to predict the **first three words of the target sentence**. The source sentence can be found under `source_test.txt`

In [0]:
!curl -O https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW3/source_test.txt
!head source_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 50587  100 50587    0     0   250k      0 --:--:-- --:--:-- --:--:--  250k
Als ich in meinen 20ern war , hatte ich meine erste Psychotherapie-Patientin .
Ich war Doktorandin und studierte Klinische Psychologie in Berkeley .
Sie war eine 26-jährige Frau namens Alex .
Und als ich das hörte , war ich erleichtert .
Meine Kommilitonin bekam nämlich einen Brandstifter als ersten Patienten .
Und ich bekam eine Frau in den 20ern , die über Jungs reden wollte .
Das kriege ich hin , dachte ich mir .
Aber ich habe es nicht hingekriegt .
Arbeit kam später , Heiraten kam später , Kinder kamen später , selbst der Tod kam später .
Leute in den 20ern wie Alex und ich hatten nichts als Zeit .


Similar to HW1, you are to predict the 100 most probable 3-gram that will begin the target sentence. The submission format will be as follows, where each word in the 3-gram will be separated by "|", and each 3-gram will be separated by space. For example, here is what an example submission might look like with 5 most-likely 3-grams (instead of 100).

```
id,word
1,Newspapers|talk|about When|I|was Researchers|call|the Twentysomethings|like|Alex But|before|long
2,That|'s|what Newspapers|talk|about You|have|robbed It|'s|realizing My|parents|wanted
3,We|forget|how We|think|about Proust|actually|links Does|any|other This|is|something
4,But|what|do And|it|'s They|'re|on My|name|is It|only|happens
```

When you print out your data, you will need to escape quotes and commas with the following command so that Kaggle does not complain. 

In [0]:
def escape(l):
    return l.replace("\"", "<quote>").replace(",", "<comma>")

You should perform your hyperparameter search/early stopping/write-up based on perplexity, not the above metric. In practice, people use a metric called [BLEU](https://www.aclweb.org/anthology/P02-1040.pdf), which is roughly a geometric average of 1-gram, 2-gram, 3-gram, 4-gram precision, with a brevity penalty for producing translations that are too short.

The test data associated with `source_test.txt` can be found [here](https://gist.githubusercontent.com/justinchiu/c4340777fa86facd820c59ff4d84c078/raw/e6ec7daba76446bc1000813680f4722060e51900/gistfile1.txt). Compute the BLEU score of your conditional de-en model with the `multi-bleu.perl` script found [here](https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl). Please submit your BLEU scores on test with your final writeup using the template provided in the repository:  https://github.com/harvard-ml-courses/nlp-template. 

